## Install Packages

In [2]:
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import pitching_stats
from pybaseball import batting_stats
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np
import pyodbc
from pybaseball import cache
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import psycopg2
from sqlalchemy import create_engine
import pyodbc

In [ ]:
param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}



### Connect to Database

In [3]:
param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}

def connect(params_dict):
    "Connecting to PostgreSQL Database Server"
    conn = None
    try:
        print("Connecting to the PostgreSQL Database")
        conn = psycopg2.connect(**params_dict)
    except (Exception, psycopg2.DataError) as error:
        print(error)
    return conn

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dict['user'],
    param_dict['password'],
    param_dict['host'],
    param_dict['database']
)
engine = create_engine(connect)

### Delete all Data in Raw_Pitch Table

In [7]:
conn.close()

NameError: name 'conn' is not defined

In [6]:
sql_query = 'DELETE FROM raw_pitch'
engine.execute(sql_query)
aprint("Data Deleted")


Data Deleted


### Create Functions for getting and appending data

In [2]:
def get_data(start_date, end_date):
    cache.enable()
    df = statcast(start_dt = start_date, end_dt = end_date)
    df["month"] = pd.to_datetime(df['game_date'])
    
    return df
    
def append_data(data):
    
    pass